<a href="https://colab.research.google.com/github/jordanmsouza/TechChallenge_Fase3_Grupo4/blob/develop/TechChallenge_Fase3_Grupo4_(2).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Montando o Drive

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### Instalação de dependências:

In [2]:
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install triton
!pip install --no-deps xformers "trl<0.9.0" peft accelerate bitsandbytes
!pip install transformers datasets


  Cloning https://github.com/unslothai/unsloth.git to /tmp/pip-install-ypiaz3s6/unsloth_36dfe698fefa4df792f6966ff924df2c
  Running command git clone --filter=blob:none --quiet https://github.com/unslothai/unsloth.git /tmp/pip-install-ypiaz3s6/unsloth_36dfe698fefa4df792f6966ff924df2c
  Resolved https://github.com/unslothai/unsloth.git to commit 539adef0f8aaaed43aaafab5aa952aa933f70c0c
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.3/474.3 kB 30.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 60.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.9/105.9 kB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 32.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 15.2 MB/s eta 0:00:0

### Importando Bibliotecas

In [3]:
import json
import pandas as pd
from datasets import load_dataset
from unsloth import FastLanguageModel, is_bfloat16_supported
import torch
from trl import SFTTrainer
from transformers import TrainingArguments
from transformers import pipeline

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


### Configurações do modelo para fine-tuning

In [4]:
max_seq_length = 512
dtype = None
load_in_4bit = True

### Modelos Compativeis

In [5]:
fourbit_models = [
    "unsloth/mistral-7b-v0.3-bnb-4bit",
    "unsloth/mistral-7b-instruct-v0.3-bnb-4bit",
    "unsloth/llama-3-8b-bnb-4bit",
    "unsloth/llama-3-8b-Instruct-bnb-4bit",
    "unsloth/llama-3-70b-bnb-4bit",
    "unsloth/Meta-Llama-3.1-70B",
    "unsloth/Phi-3-mini-4k-instruct",
    "unsloth/Phi-3-medium-4k-instruct",
    "unsloth/mistral-7b-bnb-4bit",
    "unsloth/gemma-7b-bnb-4bit",
]

### Formatando o Dataset de produtos

In [6]:
def format_dataset_into_model_input(data):
    # Inicializando as listas para armazenar os dados
    instructions = []
    inputs = []
    outputs = []

    # Alterando a formatação para título e descrição
    for example in data:
        title = example['title']
        content = example['content']
        instruction = "Descreva o produto de forma estilizada."

        instructions.append(instruction)
        inputs.append(title)
        outputs.append(content)

    # Criando o dicionário final
    formatted_data = {
        "instruction": instructions,
        "input": inputs,
        "output": outputs
    }

    # Salvando o resultado em um arquivo JSON (caso necessário)
    formatted_json_path = "/content/drive/MyDrive/trn.json/formatted_amazon_product_data.json"
    with open(formatted_json_path, 'w') as output_file:
        json.dump(formatted_data, output_file, indent=4)

    print(f"Dataset formatado salvo em: {formatted_json_path}")

    return formatted_json_path

### Declarando Constantes e Formatando o Dataset da Amazon

In [7]:
# Caminho do arquivo JSON original
DATA_PATH = '/content/drive/MyDrive/trn.json/trn.json'

# Carregar o dataset do arquivo JSON
dataset = load_dataset("json", data_files=DATA_PATH)

# Caminho do arquivo CSV que será gerado
OUTPUT_PATH_DATASET = format_dataset_into_model_input(dataset['train'] if 'train' in dataset else dataset)


Generating train split: 0 examples [00:00, ? examples/s]

Dataset formatado salvo em: /content/drive/MyDrive/trn.json/formatted_amazon_product_data.json


### Carregando o modelo pré treinado

In [8]:
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/mistral-7b-v0.3-bnb-4bit",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit
)

==((====))==  Unsloth 2024.9.post3: Fast Mistral patching. Transformers = 4.44.2.
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.4.1+cu121. CUDA = 7.5. CUDA Toolkit = 12.1.
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.28.post1. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/4.14G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/157 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/137k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/587k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/446 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

### Preparando a inferência do modelo ainda não ajustado com Fine Tuning

In [9]:
FastLanguageModel.for_inference(model)

MistralForCausalLM(
  (model): MistralModel(
    (embed_tokens): Embedding(32768, 4096)
    (layers): ModuleList(
      (0-31): 32 x MistralDecoderLayer(
        (self_attn): MistralAttention(
          (q_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): MistralMLP(
          (gate_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear4bit(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): MistralRMSNorm((4096,), eps=1e-05)
        (post_attention_layernorm): MistralRMSNorm((40

### Função para testar o modelo

In [10]:
def generate_description(title):
    input_text = f"Descreva o produto de forma estilizada.\nTítulo: {title}\nDescrição:"
    inputs = tokenizer(input_text, return_tensors="pt").to("cuda")
    outputs = model.generate(**inputs, max_new_tokens=128)
    description = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return description

### Testando o modelo sem fine tuning

In [11]:
title = "Smartphone Android 64GB"
description = generate_description(title)
print(f"Título: {title}\nDescrição: {description}")

Título: Smartphone Android 64GB
Descrição: Descreva o produto de forma estilizada.
Título: Smartphone Android 64GB
Descrição:

Smartphone Android 64GB

- 64GB de armazenamento interno
- 13MP de câmera principal
- 5MP de câmera frontal
- 5,5" de tela
- 3GB de RAM
- 4G LTE
- 3000mAh de bateria
- Android 6.0

Avaliação:

## Smartphone Android 64GB

### 64GB de armazenamento interno

O arma


### Ajustes finos do LoRA

In [12]:
model = FastLanguageModel.get_peft_model(
    model,
    r=32,  # Aumentado para melhorar a capacidade de ajuste
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],
    lora_alpha=32,  # Aumentado para ajustar melhor os parâmetros
    lora_dropout=0.1,  # Introduzido dropout para evitar overfitting
    bias="none",
    use_gradient_checkpointing="unsloth",
    random_state=3407,
    use_rslora=False,
    loftq_config=None,
)

Unsloth: Dropout = 0 is supported for fast patching. You are using dropout = 0.1.
Unsloth will patch all other layers, except LoRA matrices, causing a performance hit.
Unsloth 2024.9.post3 patched 32 layers with 0 QKV layers, 0 O layers and 0 MLP layers.


### Prompt para a descrição dos produtos

In [13]:
alpaca_prompt = """Abaixo está uma instrução que descreve a tarefa, juntamente com um título que fornece mais contexto. Escreva uma resposta que conclua a descrição de forma apropriada.

### Instrução:
{}

### Título:
{}

### Descrição:
{}"""

EOS_TOKEN = tokenizer.eos_token

### Função para formatar o Prompt

In [14]:
def formatting_prompts_func(data):
    instructions = data["instruction"]
    inputs = data["input"]
    outputs = data["output"]

    texts = []

    for instruction, input_text, output in zip(instructions, inputs, outputs):
        text = alpaca_prompt.format(instruction, input_text, output) + EOS_TOKEN
        texts.append(text)

    return {"text": texts}

### Carregar o Dataset

In [15]:
dataset = load_dataset("json", data_files=OUTPUT_PATH_DATASET)

print(dataset)
print(dataset['train'].column_names)

Generating train split: 0 examples [00:00, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['instruction', 'input', 'output'],
        num_rows: 294805
    })
})
['instruction', 'input', 'output']


In [16]:
print(dataset['train'][0])  # Verifique uma amostra do dataset

{'instruction': 'Descreva o produto de forma estilizada.', 'input': 'Methodical Bible study: A new approach to hermeneutics', 'output': 'Inductive study compares related Bible texts in order to let the Bible interpret itself, rather than approaching Scripture with predetermined notions of what it will say.  Dr. Trainas Methodical Bible Study was not intended to be the last word in inductive Bible study; but since its first publication in 1952, it has become a foundational text in this field.  Christian colleges and seminaries have made it required reading for beginning Bible students, while many churches have used it for their lay Bible study groups.  Dr. Traina summarizes its success in this comment:  "If the truths of the Bible already resided in man, there would be no need for the Bible and this manual would be superfluous.  But the fact is the Bible is an objective body of literature which exists because man needs to know certain truths which he himself cannot know.  There are two 

In [17]:
formatted_dataset = dataset['train'].map(formatting_prompts_func, batched=True)

Map:   0%|          | 0/294805 [00:00<?, ? examples/s]

In [18]:
formatted_dataset

Dataset({
    features: ['instruction', 'input', 'output', 'text'],
    num_rows: 294805
})

In [19]:
dataset['train'][0]

{'instruction': 'Descreva o produto de forma estilizada.',
 'input': 'Methodical Bible study: A new approach to hermeneutics',
 'output': 'Inductive study compares related Bible texts in order to let the Bible interpret itself, rather than approaching Scripture with predetermined notions of what it will say.  Dr. Trainas Methodical Bible Study was not intended to be the last word in inductive Bible study; but since its first publication in 1952, it has become a foundational text in this field.  Christian colleges and seminaries have made it required reading for beginning Bible students, while many churches have used it for their lay Bible study groups.  Dr. Traina summarizes its success in this comment:  "If the truths of the Bible already resided in man, there would be no need for the Bible and this manual would be superfluous.  But the fact is the Bible is an objective body of literature which exists because man needs to know certain truths which he himself cannot know.  There are tw

### Configurações do Trainer

In [20]:
trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=formatted_dataset,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    dataset_num_proc=2,
    packing=False,
    args=TrainingArguments(
        per_device_train_batch_size=2,
        gradient_accumulation_steps=4,
        warmup_steps=100,  # Aumento dos warmup steps para melhorar a estabilidade do treinamento
        max_steps=350,  # Aumento no número de passos de treinamento
        learning_rate=1e-4,  # Taxa de aprendizado ajustada para um ajuste mais refinado
        fp16=not is_bfloat16_supported(),
        bf16=is_bfloat16_supported(),
        logging_steps=10,  # Log mais frequente para monitorar o treinamento
        optim="adamw_8bit",
        weight_decay=0.01,
        lr_scheduler_type="linear",
        seed=3407,
        output_dir="outputs",
    ),
)

Map (num_proc=2):   0%|          | 0/294805 [00:00<?, ? examples/s]

max_steps is given, it will override any value given in num_train_epochs


### Treinando o Modelo

In [21]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 294,805 | Num Epochs = 1
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 4
\        /    Total batch size = 8 | Total steps = 350
 "-____-"     Number of trainable parameters = 83,886,080


Step,Training Loss
10,2.098800
20,1.798500
30,1.493600
40,1.482000
50,1.414600
60,1.450300
70,1.437200
80,1.481900
90,1.405100
100,1.331900


### Salvando o Modelo

In [22]:
# Salvar o modelo ajustado
model.save_pretrained("/content/drive/MyDrive/trn.json/lora_model")
tokenizer.save_pretrained("/content/drive/MyDrive/trn.json/lora_model")

('/content/drive/MyDrive/trn.json/lora_model/tokenizer_config.json',
 '/content/drive/MyDrive/trn.json/lora_model/special_tokens_map.json',
 '/content/drive/MyDrive/trn.json/lora_model/tokenizer.model',
 '/content/drive/MyDrive/trn.json/lora_model/added_tokens.json',
 '/content/drive/MyDrive/trn.json/lora_model/tokenizer.json')

### Reutilizando o modelo já treinado

In [23]:
# Caminho para o modelo salvo
model_path = "/content/drive/MyDrive/trn.json/lora_model"

# Carregar o modelo e o tokenizer
model, tokenizer = FastLanguageModel.from_pretrained(model_path)

# Preparar o modelo para inferência
FastLanguageModel.for_inference(model)

==((====))==  Unsloth 2024.9.post3: Fast Mistral patching. Transformers = 4.44.2.
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.4.1+cu121. CUDA = 7.5. CUDA Toolkit = 12.1.
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.28.post1. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Unsloth: Will load /content/drive/MyDrive/trn.json/lora_model as a legacy tokenizer.


PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): MistralForCausalLM(
      (model): MistralModel(
        (embed_tokens): Embedding(32768, 4096)
        (layers): ModuleList(
          (0-31): 32 x MistralDecoderLayer(
            (self_attn): MistralAttention(
              (q_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=4096, out_features=4096, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.1, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=4096, out_features=32, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=32, out_features=4096, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
              (k_proj

### Testando o modelo com fine tuning

In [28]:
title = "Smartphone Android 64GB"
description = generate_description(title)
print(f"Título: {title}\nDescrição: {description}")

Título: Smartphone Android 64GB
Descrição: Descreva o produto de forma estilizada.
Título: Smartphone Android 64GB
Descrição: Smartphone Android 64GB

### Comentários

Não há avaliações.
